In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import time

from lightglue import LightGlue, SuperPoint, viz2d, match_pair
from lightglue.utils import rbd

from lac.slam.feature_tracker import FeatureTracker
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data, load_stereo_images, load_images
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

# Load some data


In [ ]:
# data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset0"
# data_path = "/home/shared/data_raw/LAC/runs/stereo_lights1.0_map1_preset1"
# data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
data_path = "../../../output/NavAgent/2025-05-09_05-20-06"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

In [ ]:
# left_imgs, right_imgs = load_stereo_images(data_path)
images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs = images["FrontLeft"]
right_imgs = images["FrontRight"]

In [ ]:
tracker = FeatureTracker(cam_config)

# Extract features


In [ ]:
image = left_imgs[100]

feats = tracker.extract_feats(image)
feats = rbd(feats)

In [ ]:
kps = feats["keypoints"]
good_kps = kps[feats["keypoint_scores"] > 0.05]
print(f"Num keypoints: {len(kps)}, {len(good_kps)}")

In [ ]:
viz2d.plot_images([image])
viz2d.plot_keypoints([kps], colors=["red"], ps=10)
viz2d.plot_keypoints([good_kps], colors=["lime"], ps=10)

# Feature matching


## Stereo


In [ ]:
frame = 700

feats1 = tracker.extract_feats(left_imgs[frame])
feats2 = tracker.extract_feats(right_imgs[frame])
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

viz2d.plot_images([left_imgs[frame], right_imgs[frame]], pad=0.0)
viz2d.plot_matches(points1, points2, color="lime", lw=0.2)

## Different frames


In [ ]:
frame1 = 1300
frame2 = 1400
img1 = left_imgs[frame1]
img2 = left_imgs[frame2]

feats1 = tracker.extract_feats(img1)
feats2 = tracker.extract_feats(img2)
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

viz2d.plot_images([img1, img2])
viz2d.plot_matches(points1, points2, lw=0.2)

# LightGlue Tracking


In [ ]:
prev_img = left_imgs[1500]
next_img = left_imgs[1502]

prev_feats = tracker.extract_feats(prev_img)
next_feats = tracker.extract_feats(next_img)

matches = tracker.match_feats(prev_feats, next_feats)
points_prev = prev_feats["keypoints"][0][matches[:, 0]]
points_next = next_feats["keypoints"][0][matches[:, 1]]

In [ ]:
len(matches)

In [ ]:
matches = tracker.match_feats(prev_feats, next_feats, min_score=0.9)
len(matches)

In [ ]:
points0 = points_prev.cpu().numpy()
points1 = points_next.cpu().numpy()

plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for i in range(len(matches)):
    plt.plot([points0[i, 0], points1[i, 0]], [points0[i, 1], points1[i, 1]], color="lime")
plt.axis("off")
plt.show()

# OpenCV LK Optical Flow


In [ ]:
# Opencv optical flow
prev_img = left_imgs[1500]
next_img = left_imgs[1502]

prev_pts = kps.cpu().numpy()

lk_params = dict(
    winSize=(21, 21),
    maxLevel=3,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03),
    minEigThreshold=1e-4,
)

# next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_pts, None, **lk_params)
next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_pts, None)
next_pts_tracked = next_pts[status.squeeze() == 1]
prev_pts_tracked = prev_pts[status.squeeze() == 1]

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for new, old in zip(next_pts_tracked, prev_pts_tracked):
    a, b = new.ravel()
    c, d = old.ravel()
    plt.arrow(c, d, a - c, b - d, color="lime", head_width=1, head_length=2, linewidth=1)
plt.show()

# Feature tracker class


In [ ]:
from lac.slam.feature_tracker import FeatureTracker, prune_features

In [ ]:
tracker = FeatureTracker(cam_config)

start_idx = 80
tracker.initialize(poses[start_idx], left_imgs[start_idx], right_imgs[start_idx])

In [ ]:
idx = start_idx
n_frames = 100

start_time = time.time()
for i in range(n_frames):
    idx += 2
    if i % 10 == 0:
        tracker.track_keyframe(poses[idx], left_imgs[idx], right_imgs[idx])
    else:
        tracker.track(left_imgs[idx])

print(f"Avg time per frame: {(time.time() - start_time) / n_frames}")

In [ ]:
tracker.max_id

In [ ]:
len(tracker.track_ids)

In [ ]:
frame1 = 1300
frame2 = 1400
img1 = left_imgs[frame1]
img2 = left_imgs[frame2]

feats1 = tracker.extract_feats(img1)
feats2 = tracker.extract_feats(img2)
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

viz2d.plot_images([img1, img2])
viz2d.plot_matches(points1, points2, lw=0.2)